<a href="https://colab.research.google.com/github/laurilaatu/malis/blob/Network_Architecture/MALIS_Project_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MALIS 2019 EURECOM
## Course Project

This is version zero of our semester project. We will load and prepare the data. Train a CNN to to extract features and use an ANN or SVM to classify the images.

### Description

**Task 1**: **bold text** Project definition for Group 34
	By:
Lauri Laatu
Marvin Mouroum


**•Context & problem definition**

Classifying images is a huge topic in machine learning. The goal for our project is to be able to classify images to multiple classes. More specifically we want to investigate if a simple convolutional neural network is able to classify internet memes correctly. As such our it is a supervised learning problem.


**•Methodology**

Convolution for feature extraction
RGB images will be fetched from a database
preprocessing to optimize image dimensions
convolution over three color channels of the images
exporting a feature vector containing compressed information about the image
ANN or SVM for classification based on extracted features
feature vectors as labeled input
supervised multiclass classification problem
dataset usage: initially we plan to start with n-fold cross validation and 10% testing
classification in predefined meme categories


**•Data**

The data will be acquired from the internet and labeled by hand if the data is unlabelled. Goal is to find around a thousand images per class.


# Dataset
Loading data from the dataset

In [0]:
import os.path
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import io
from googleapiclient.http import MediaIoBaseDownload
from google.colab import drive as gdrive
from pydrive.auth import GoogleAuth

from IPython.display import Image

In [0]:
auth.authenticate_user()

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
gdrive.mount('/content/drive')

### Copy raw pepe images to local directory

In [6]:
!mkdir  dataset

file_list = drive.ListFile({'q': "'10DpgRQAxqIU0TLYCruSGPaj3DfRzh6TE' in parents and trashed=false"}).GetList()

root = "/content/drive/My Drive/MALIS_Project/DataSet/rare-pepes/"
i = 0

#!cp "/content/drive/My Drive/MALIS_Project/DataSet/rare-pepes/001 - OdrldTF.png" "dataset/pepe_00.png"

for file in file_list:
  target = "'dataset/pepe_"+str(i)+".png'"
  src    = "'"+root+file['title']+"'"
  #print(src)
  #print(target)
  !cp $src $target
  i+=1

!ls -1 dataset



pepe_0.png
pepe_1000.png
pepe_1001.png
pepe_1002.png
pepe_1003.png
pepe_1004.png
pepe_1005.png
pepe_1006.png
pepe_1007.png
pepe_1008.png
pepe_1009.png
pepe_100.png
pepe_1010.png
pepe_1011.png
pepe_1012.png
pepe_1013.png
pepe_1014.png
pepe_1015.png
pepe_1016.png
pepe_1017.png
pepe_1018.png
pepe_1019.png
pepe_101.png
pepe_1020.png
pepe_1021.png
pepe_1022.png
pepe_1023.png
pepe_1024.png
pepe_1025.png
pepe_1026.png
pepe_1027.png
pepe_1028.png
pepe_1029.png
pepe_102.png
pepe_1030.png
pepe_1031.png
pepe_1032.png
pepe_1033.png
pepe_1034.png
pepe_1035.png
pepe_1036.png
pepe_1037.png
pepe_1038.png
pepe_1039.png
pepe_103.png
pepe_1040.png
pepe_1041.png
pepe_1042.png
pepe_1043.png
pepe_1044.png
pepe_1045.png
pepe_1046.png
pepe_1047.png
pepe_1048.png
pepe_1049.png
pepe_104.png
pepe_1050.png
pepe_1051.png
pepe_1052.png
pepe_1053.png
pepe_1054.png
pepe_1055.png
pepe_1056.png
pepe_1057.png
pepe_1058.png
pepe_1059.png
pepe_105.png
pepe_1060.png
pepe_1061.png
pepe_1062.png
pepe_1063.png
pepe_1064.png
p

### Resizing Image so it can be used by the CNN

In [0]:
def resize_img(img,destination):
  # Create square images from pepes by adding black margins preserving original aspect ratio
  #Importing modules opencv + numpy
  import cv2
  import numpy as np

  #Reading an image (you can use PNG or JPG)
  img = cv2.imread(img)

  if img is None:
    return False

  #Getting the bigger side of the image
  s = max(img.shape[0:2])

  #Creating a dark square with NUMPY  
  f = np.zeros((s,s,3),np.uint8)

  #Getting the centering position
  ax,ay = (s - img.shape[1])//2,(s - img.shape[0])//2

  #Pasting the 'image' in a centering position
  f[ay:img.shape[0]+ay,ax:ax+img.shape[1]] = img

  #Showing results (just in case) 
  #cv2.imshow("IMG",f)
  #A pause, waiting for any press in keyboard
  #cv2.waitKey(0)

  #Saving the image
  f = cv2.resize(f,(227,227),interpolation=cv2.INTER_AREA)
  cv2.imwrite(destination,f)
  cv2.destroyAllWindows() 
  return True

In [0]:
!mkdir resized
for i in range(0,1273):
  path =   "dataset/pepe_" + str(i) + ".png"
  _path = "'resized/pepe_" + str(i) + ".png'"
  prev_dest = "resized/pepe_" + str(i) + ".png"
  destination = "'/content/drive/My Drive/MALIS_Project/DataSet/resized_pepe/pepe_" + str(i) + ".png'"
  if not resize_img(path,prev_dest):
    print("error with", i)
  !cp $_path $destination 
  if(i%50 == 0):
    print(i)

### Prepare the Not Pepe Images

In [0]:
file_list = drive.ListFile({'q': "'1UBhhkryHE302KDQ8ufDvkfv7vozMZfli' in parents and trashed=false"}).GetList()

!mkdir dataset

root = "/content/drive/My Drive/MALIS_Project/DataSet/not-pepes/"
i = 0
rootfolder = "'"+root+"'"

#!cp "/content/drive/My Drive/MALIS_Project/DataSet/rare-pepes/001 - OdrldTF.png" "dataset/pepe_00.png"

for file in file_list:
  target = "dataset/not_pepe_"+str(i)+"."+file['title'].split('.')[-1]
  src    =  root+file['title']
  _src   = "'"+target+"'"  
  _target = "'/content/drive/My Drive/MALIS_Project/DataSet/resized_not_pepe/not_pepe_"+str(i)+"."+file['title'].split('.')[-1]+"'"
  
  if not resize_img(src,target):
    print("error with", i)
    continue
  !cp $_src $_target
  i+=1

mkdir: cannot create directory ‘dataset’: File exists


In [0]:
!ls '/content/drive/My Drive/MALIS_Project/DataSet/not-pepes/'

## Network

Defining the Network class

In [0]:
class Net(Module):   
    
    def __init__(self,name='PepeNN'):
        super(PepeNN, self).__init__()
        self.T = 120
        
        self.name = name
        
        self.training_epochs = 0
        
        self.lr_history       = []
        self.accuracy_history = []
        self.loss_history     = []
        
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
        #torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)

        self.conv1 = nn.Conv2D(3,95,(11,11),(4,4),(3,3),0)
        self.batchnorm1 = nn.BatchNorm2d(95)
        self.conv2 = nn.Conv2D(95,120,(5,5),(2,3),(1,2),0)
        self.batchnorm2 = nn.BatchNorm2d(120)

        self.pooling1 = nn.MaxPool2d((5,5), stride=(1,3), padding=(0,0), dilation=0)

        self.conv3 = nn.Conv2D(120,140,(5,5),(1,1),(0,0),0)
        self.batchnorm3 = nn.BatchNorm2d(140)
        self.conv4 = nn.Conv2D(140,180,(3,3),(1,1),(0,0),0)
        self.batchnorm4 = nn.BatchNorm2d(180)

        self.pooling2 = nn.MaxPool2d((3,3), stride=(1,1), padding=(0,0), dilation=0)
        
        #fully connected at the end
        self.fc1 = torch.nn.Linear(in_features=22, out_features=11)
        self.fc2 = torch.nn.Linear(in_features=11, out_features=2)
    
        
        # robust weight initialization
        torch.nn.init.xavier_normal_(self.conv1.weight)
        torch.nn.init.xavier_normal_(self.conv2.weight)
        torch.nn.init.xavier_normal_(self.conv3.weight)
        torch.nn.init.xavier_normal_(self.conv4.weight)

        torch.nn.init.xavier_normal_(self.fc1.weight)
        torch.nn.init.xavier_normal_(self.fc2.weight)
      

        
    def forward(self, x):

        # first set of CNNs and then a max pool
        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = F.relu(x)
        
        x = self.conv2(x)
        x = self.batchnorm2(x)
        x = F.relu(x)
        
        x = self.pooling1(x)
        
        # second set
        
        x = self.conv3(x)
        x = self.batchnorm3(x)
        x = F.relu(x)
        
        x = self.conv4(x)
        x = self.batchnorm4(x)
        x = F.relu(x)
        
        x = self.pooling2(x)
        
        batch_size, timesteps, C, H, W = x.size()
        
        x = x.view(batch_size, timesteps, W)
        
        return x
      
    def save(self,root='gdrive/My\ Drive/MALIS_Project'):
      !mkdir $root
  
      path = root.replace('\\','') + '/' + self.name + '.pickle'
    
      print("\nsaving network under:\n",path)
      
      with open(path, 'wb') as f:
        pickle.dump(self, f)
        
    def next_epoch(self,lr,acc_hist,loss_hist):
      self.training_epochs += 1
      self.lr_history.append(lr)
      self.accuracy_history.append(acc_hist)
      self.loss_history.append(loss_hist)
    
    def reset(self):
      self.training_epochs = 0
      self.lr_history = []
      self.accuracy_history = []
      self.loss_history = []